In [ ]:
#this is the code for identifying  keystone species

In [1]:
import torch
import numpy as np
import sys, copy, math, time, pdb
import os.path
import random
import pdb
import csv
import argparse
import itertools
from itertools import permutations, product
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchdiffeq import odeint
import itertools

https://github.com/spxuw/DKI/blob/main/DKI.py